# **Predicting Subscription Status Based on player characteristics**

### Data Science 100 Project - Group 10

- Kyle Nguyen (76276393)
- Jiayin Wang (47186200)
- Clianta Anindya (78508892)


### **Introduction**

#### **(1) Background Information**
In recent years, the gaming industry has increasingly relied on data to better understand player behavior and improve user engagement.
One area of interest is predicting which players are most likely to subscribe to game-related newsletters, as this can inform targeted recruitment strategies, optimize resource allocation, and improve community building around gaming platforms.

This project is based on data collected by a research group at UBC, led by Professor Frank Wood. The group operates a Minecraft research server (_plaicraft.ai_) where they track how users interact within the virtual world. The goal of the broader research initiative is to study player behavior at scale and to identify actionable insights for improving the management and outreach of such servers.

Our group is addressing Question 1 from the project brief: **What player characteristics and behaviours are most predictive of subscribing to a game-related newsletter, and how do these features differ between various player types?**

To explore this, we focus specifically on whether player age, total hours played and session count can predict whether a player subscribed to the newsletter. The first two variables are available in the **_players.csv_** dataset (**_Age_** and **_played_hours_**, respectively) while the session count variable will be taken from the sum of the number of rows in the **_sessions.csv_** dataset for each hashed email that is unique per player.

Understanding how these player characteristics relate to newsletter subscription behavior will help stakeholders improve how they engage different player types, and may also provide insights into motivations behind long-term or more invested gameplay.

#### **(2) Questions**
- **Broad question**: What player characteristics and behaviours are most predictive of subscribing to a game-related newsletter, and how do these features differ between various player types?
- **Specific question**: Can **_age_**, **_played_hours_** **_session_count_** predict **_subscribe_** in the data set?
#### **(3) Description of data sets used**
We will be using the **players.csv** and **sessions.csv** datasets.

The **players.csv** set has 196 rows, each representing one players, uniquely identified by _hashedEmail_.
There are 7 variables in the **players.csv** data set:
* _experience_: character, indicating player's experience level
* _hashedEmail_: character, unique identifier for each player
* _name_: character, player's name
* _gender_: character, player's gender
* _played_hours_: double, total hour the player has played
* _Age_: double, player's age
* _subscribe_: logical, whether the player subscribes the newsletter

The **sessions.csv** set has 1535 rows, each representing one play session.
There are 5 variables in the **session.csv** data set:
* _hashedEmail_: character, unique identifier connecting session to a player
* _start_time_: character, start time in DD/MM/YYYY HH:MM format
* _end_time_: character, end time in DD/MM/YYYY HH:MM format
* _original_start_time_: double, unix timestamp of start time
* _original_end_time_: double, unix timestamp of end time

### **Methods and Results**

In [1]:
# Run this cell first to load the libraries and set row view limit before continuing
# Suppress package startup messages with suppressMessages() for cleanliness
suppressMessages(library(repr))
suppressMessages(library(tidyverse))
suppressMessages(library(tidymodels))

options(repr.matrix.max.rows = 6)

In [2]:
# Read the csv files to their corresponding dataframe variable
# Suppress read_csv() messages with suppressMessages() for cleanliness
players <- suppressMessages(read_csv("data/players.csv"))
sessions <- suppressMessages(read_csv("data/sessions.csv"))

In [6]:
# Group by hashedEmail and summarize the count of rows aggregated per group
sessions_formatted <- group_by(sessions, hashedEmail) |>
    summarize(session_count = n())

# Left join players to sessions_formatted via hashedEmail to retain all the players rows
# Replace all blank session_counts with 0 (all players with blank session_counts have a 0 for played_hours)
# Rename 'Age' to 'age' for lowercase consistency in the variables
# Filter only the desired exploratory variables and response variable
# Remove rows with a blank age (there are only 2 NAs in the 'age' column)
players_tidy <- left_join(players, sessions_formatted, by = "hashedEmail") |>
    mutate(session_count = replace_na(session_count, 0)) |>
    rename(age = Age) |>
    select(age, played_hours, session_count, subscribe) |>
    filter(!is.na(age))

head(players_tidy)

age,played_hours,session_count,subscribe
<dbl>,<dbl>,<int>,<lgl>
9,30.3,27,TRUE
17,3.8,3,TRUE
17,0.0,1,FALSE
21,0.7,1,TRUE
21,0.1,1,TRUE
17,0.0,0,TRUE


### **Discussions**